In [1]:
import os

In [2]:
import nltk

In [3]:
import math

In [4]:
class Model():
    def __init__(self):
        self.classes = {}
               
    def trainNBClassifier(self):
        logprob={}        
        bigdoc={}
        loglikelihood={}
        for classe in self.classes:
            text=''
            
            number_of_documents = sum(map(lambda classes:int(classes),self.classes.values())) 
            number_of_doc_in_class = self.classes[classe]
            
            logprob[classe] = math.log(number_of_doc_in_class/number_of_documents)
        
            vector_of_words = self.extractVocabulary()
            
            files=os.listdir('./dataset/train/'+classe)
            for doc in files:
                text += self.read_file("./dataset/train/"+classe+"/"+doc)
            
            bigdoc[classe]= text
            
            words_of_classe = nltk.tokenize.word_tokenize(bigdoc[classe])
            
            for word in vector_of_words:
                loglikelihood[word,classe] = (bigdoc[classe].count(word) +1 / (len(words_of_classe)+len(vector_of_words)))
        
        print(vector_of_words)
        print(loglikelihood)
        
        self.save(str(loglikelihood))
        
        return (logprob, loglikelihood, vector_of_words )
        
       
    def getTrainingCorpus(self):
        folders=os.listdir('./dataset/train')
        for classe in folders:
            number_of_docs = len(os.listdir('./dataset/train/'+classe))
            self.classes[classe]=number_of_docs  
        
    
    def extractVocabulary(self):
        bigtext=''
        folders=os.listdir('./dataset/train')
        for classe in folders:            
            files=os.listdir('./dataset/train/'+classe)
            for doc in files:
                bigtext += self.read_file("./dataset/train/"+classe+"/"+doc)
        
        sentences = nltk.sent_tokenize(bigtext)        
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            words= [w for w in words if w.isalpha()] # remove all tokens that are not alphabetic
            #word_list = [w for w in words if not w in nltk.corpus.stopwords.words('english')]
           
        return set(words)
            
    
    def read_file(self,filename):
        with open (filename, 'r') as file:
            text = file.read()
        return text    
    
    def save (self, classifier):
        f = open('NBclassifier','w')
        f.write(classifier)        
        f.close()
    

In [5]:
if __name__ == "__main__":
    model = Model()  
    model.getTrainingCorpus()
    model.trainNBClassifier()

NameError: name 'folders' is not defined